In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import sys
sys.path.append('../')

from src.functions import Functions
from src.modeling import Modeling

import warnings
warnings.filterwarnings('ignore')

In [2]:
features = pd.read_csv('../data/raw_data/features.csv')
sales_train = pd.read_csv('../data/raw_data/train.csv')
stores = pd.read_csv('../data/raw_data/stores.csv')
sales_test = pd.read_csv('../data/raw_data/test.csv')

In [3]:
df_all_train = sales_train.merge(features, how="left", on=["Store","Date","IsHoliday"]).merge(stores, how="left", on=["Store"])
df_all_test = sales_test.merge(features, how="left", on=["Store","Date","IsHoliday"]).merge(stores, how="left", on=["Store"])

df_all_train['train_or_test'] = 'train'
df_all_test['train_or_test'] = 'test'
merged_df = pd.concat([df_all_train, df_all_test], sort=False)

merged_df = merged_df.sort_values(by=['Store','Dept','Date'], axis=0).reset_index().drop(columns='index')
merged_df['Date'] = Functions.datetimeConverter(merged_df, 'Date')

In [4]:

Superbowl = pd.DataFrame({
  'holiday' : 'superbowl',
  'ds' : pd.to_datetime(['2010-02-12','2011-02-11','2012-02-10','2013-02-08'])
  })

Laborday = pd.DataFrame({
  'holiday' : 'laborday',
  'ds' : pd.to_datetime(['2010-09-10','2011-09-09','2012-09-07','2013-09-06'])
  })

Thanksgiving = pd.DataFrame({
  'holiday' : 'thanksgiving',
  'ds' : pd.to_datetime(['2010-11-26','2011-11-25','2012-11-23','2013-11-29'])
  })

Christmas = pd.DataFrame({
  'holiday' : 'christmas',
  'ds' : pd.to_datetime(['2010-12-31','2011-12-30','2012-12-28','2013-12-27'])
  })

BlackFriday = pd.DataFrame({
  'holiday' : 'blackfriday',
  'ds' : pd.to_datetime(['2010-11-26','2011-11-25','2012-11-23','2013-11-29'])
  })

PreChristmas = pd.DataFrame({
  'holiday' : 'prechristmas',
  'ds' : pd.to_datetime(['2010-12-23','2010-12-24','2011-12-23','2011-12-24','2012-12-23','2012-12-24','2013-12-23','2013-12-24'])
  })

holidays = pd.concat((Superbowl, Laborday, Thanksgiving, Christmas, BlackFriday, PreChristmas))

In [5]:
init_ts = merged_df[['Store', 'Dept', 'Date', 'Weekly_Sales', 'train_or_test']]
init_ts = init_ts.set_index('Date')
ts_store_dept = init_ts[['Store', 'Dept']].drop_duplicates().reset_index(drop=True).values.tolist()

ts = []
idx = pd.date_range('2/5/2010', '10/26/2012', freq='W-FRI')

for store, dept in ts_store_dept:
    tmp_ts = init_ts[(init_ts['Store'] == store) & (init_ts['Dept'] == dept) & (init_ts['train_or_test'] == 'train')].reindex(idx)
    values = {'Store': store, 'Dept': dept, 'Weekly_Sales': 0, 'train_or_test': 'train'}
    tmp_ts = tmp_ts.fillna(value=values)
  
    ts.append(tmp_ts)
    ts.append(init_ts[(init_ts['Store'] == store) & (init_ts['Dept'] == dept) & (init_ts['train_or_test'] == 'test')])

ts = pd.concat(ts)

In [6]:
from fbprophet import Prophet
ts_p = ts.copy()
ts_p = ts_p.reset_index().rename(columns = {'index':'Date'})
ts_forecast = []
store_dept = sales_test[['Store','Dept']].drop_duplicates().values.tolist()

for store, dept in store_dept:
  
    print ('Store No: ', store, ' Dept No:', dept)

    df = ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'train')][['Date', 'Weekly_Sales']]

    df = df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})\

    if len(df) >= 5:
        prophet = Prophet(holidays=holidays)
        prophet.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        prophet.fit(df)

        df_forecast = prophet.make_future_dataframe(periods=39, freq='W-FRI')

    
        df_forecast = prophet.predict(df_forecast)

        df_forecast = pd.merge(ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')][['Store', 'Dept', 'Date']], 
                               df_forecast[['ds', 'yhat']], 
                               left_on='Date', right_on='ds', how = 'left')[['Store', 'Dept', 'Date', 'yhat']]

    else:

        forecast = np.repeat(np.average(ts_p[(ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'train')]['Weekly_Sales']), 
                             len(ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')]))

        df_forecast = ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')][['Store', 'Dept', 'Date']]

        df_forecast['yhat'] = forecast

    ts_forecast.append(df_forecast)

ts_forecast = pd.concat(ts_forecast)
ts_forecast['Store'] = ts_forecast['Store'].astype(int)
ts_forecast['Dept'] = ts_forecast['Dept'].astype(int)
ts_forecast['Id'] = ts_forecast['Store'].astype(str) + '_' +  ts_forecast['Dept'].astype(str) + '_' +  ts_forecast['Date'].astype(str)
ts_forecast = ts_forecast.rename(columns = {'yhat': 'Weekly_Sales'})
ts_forecast = ts_forecast[['Id', 'Weekly_Sales']]

Importing plotly failed. Interactive plots will not work.


Store No:  1  Dept No: 1


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store No:  1  Dept No: 2


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Store No:  45  Dept No: 95


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store No:  45  Dept No: 97


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Store No:  45  Dept No: 98


In [7]:
ss = pd.read_csv('../data/raw_data/sampleSubmission.csv')
ss.loc[:, 'Weekly_Sales'] = list(ts_forecast.iloc[:, 1])
ss.to_csv('../files/submissions/prophet_holidays_addseason.csv', index=False)